# Geocoding hiking trails

In [36]:
import warnings
warnings.filterwarnings('ignore')
import os

from perdido.geoparser import Geoparser
import gpxpy
import folium
import geojson

from spacy import displacy

In [ ]:
txt_path = os.path.join("data", "visorando-2013", "txt")
gpx_path = os.path.join("data", "visorando-2013", "gpx")

In [ ]:
''' function get_bbox() returns the bounding box of a given GPS trace '''
def get_bbox(gpx_filename):
    latitudes = []
    longitudes = []
    elevations = []
    gpx = gpxpy.parse(open(gpx_filename, 'r'))
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                latitudes.append(point.latitude)
                longitudes.append(point.longitude)
                elevations.append(point.elevation)
    return str(min(longitudes))+' '+str(min(latitudes))+' '+str(max(longitudes))+' '+str(max(latitudes))


''' function display_map_gpx() display the map using the folium library '''
def display_map_gpx(gpx_filename, json_data=None):
    gpx = gpxpy.parse(open(gpx_filename, 'r'))
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
    ave_lat = sum(p[0] for p in points)/len(points)
    ave_lon = sum(p[1] for p in points)/len(points) 

    m = folium.Map(location=[ave_lat, ave_lon], zoom_start=12)
    folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(m)
    
    if json_data is not None:
        coords = list(geojson.utils.coords(json_data)) 

        if len(coords) > 0:

            print(str(len(coords))+" records found in gazetteer:")
            folium.GeoJson(json_data, name='Toponyms', tooltip=folium.features.GeoJsonTooltip(fields=['id', 'name', 'source'], localize=True)).add_to(m)
        else:
            print("Sorry, no records found in gazetteer for geocoding!")

    display(m)


In [ ]:
filename = "1e_jour_de_champagny_le_haut_au_refuge_d"
txt_file = os.path.join(txt_path, filename + ".txt")
gpx_file = os.path.join(gpx_path, filename + ".gpx")

file = open(txt_file, "r")

# get the textual content from file
content = ""
for paragraph in file:
    content += paragraph
content = content.strip()
    
# get the bbox from the GPS file
bbox = get_bbox(gpx_file)

print('content: ', content)
print('bbox: ',bbox)

Au départ du Refuge du Bois dans la belle vallée encaissée de Champagny-le-Haut et avec en point de mire la Grande Motte, passer devant l'église et au carrefour prendre à droite. Arrivé près du torrent, le longer jusqu'au hameau de Friburge.
Traverser le pont, puis à 200m prendre le petit sentier à gauche. Suivre ainsi le torrent jusqu'au pont du Laisonnay d'en Bas qu'il faut traverser. Passer devant le parking jusqu'au prochain pont qui enjambe la cascade de Py et prendre tout de suite à gauche le sentier abrupt qui grimpe. La pente est raide puisque l'on attaque un fort dénivelé de 1000m.
Suivre tout le temps la cascade puis continuer après le refuge de Plaisance, franchir la cascade et contourner le cirque du Cul du Nant par la droite. Poursuivre sur ce sentier qui passe à proximité du lac du Plan Séry. Plus tard, parvenu à un Y avec vue sur le lac de La Plagne en contrebas, ne pas descendre vers le refuge du lac mais continuer par la droite.
Le chemin reste sur le même dénivelé dur

## Geoparsing (NER and geocoding) with Perdido

https://github.com/ludovicmoncla/perdido

In [ ]:
geoparser = Geoparser()
doc = geoparser(content)

In [37]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

In [22]:
display_map_gpx(gpx_file, doc.geojson)

12 records found in gazetteer:


In [23]:
geoparser = Geoparser(bbox=bbox)
doc = geoparser(content)

display_map_gpx(gpx_file, doc.geojson)

12 records found in gazetteer:


## NER with spaCy

https://spacy.io


```bash
pip install -U pip setuptools wheel
pip install -U 'spacy[apple]'
python -m spacy download fr_core_news_sm
```



In [38]:
import spacy
nlp = spacy.load("fr_core_news_sm")
doc = nlp(content)

displacy.render(doc, style="ent", jupyter=True)

In [30]:
locations = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
locations

['Refuge du Bois',
 'Champagny-le-Haut',
 'Grande Motte',
 'Traverser',
 'pont du Laisonnay',
 'Bas',
 'Py',
 'refuge de Plaisance',
 'Cul du Nant',
 'lac du Plan Séry',
 'lac de La Plagne',
 'lac de la Plagne',
 'lac de Gratteleu',
 'refuge du Palet']

## Geocoding with Perdido

By default, Perdido uses the OpenStreetMap Nominatim API.

In [32]:
from perdido.geocoder import Geocoder

geocoder = Geocoder(bbox=bbox)
doc = geocoder(locations)

display_map_gpx(gpx_file, doc.geojson)

13 records found in gazetteer:


In [33]:
doc.geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [6.744398, 45.459549]},
   'properties': {'id': 0,
    'name': 'Refuge du Bois',
    'sourceName': 'Refuge du Bois, Rue Michel Ruffier-Lanche, Le Bois, Champagny-en-Vanoise, Albertville, Savoie, Auvergne-Rhône-Alpes, France métropolitaine, 73350, France',
    'type': 'alpine_hut',
    'country': 'France',
    'source': 'nominatim'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [6.693793, 45.464146]},
   'properties': {'id': 0,
    'name': 'Champagny-le-Haut',
    'sourceName': "Champagny, Rue des Hauts du Crey, L'Envers de Reclaz, La Traye, Champagny-en-Vanoise, Albertville, Savoie, Auvergne-Rhône-Alpes, France métropolitaine, 73350, France",
    'type': 'gondola',
    'country': 'France',
    'source': 'nominatim'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [6.86988, 45.410791]},
   'properties': {'id': 0,
    'name': 'G